In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy import signal
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
import random

In [ ]:
print("All modules imported successfully.")

In [17]:
data = pd.read_csv('emotions.csv')

In [18]:
label_mapping = {'NEGATIVE': 0, 'NEUTRAL': 1, 'POSITIVE': 2}
data['label'] = data['label'].map(label_mapping)

In [19]:
emotion_counts = data['label'].value_counts()

# Define emotional labels
emotional_labels = {0: 'NEGATIVE', 1: 'NEUTRAL', 2: 'POSITIVE'}

# Map numerical labels to emotional labels
emotion_labels = [emotional_labels[label] for label in emotion_counts.index]

In [20]:
emotions = data['label'].unique()

In [21]:
from scipy.stats import ttest_ind

# Define colors for significant and non-significant features
significant_color = 'green'
non_significant_color = 'red'

# Create a dictionary to store the number of significant and non-significant features for each emotion
num_features = {emotion: {'significant': 0, 'non-significant': 0} for emotion in emotions}

# Perform t-tests and count significant features for each emotion
for emotion in emotions:
    subset = data[data['label'] == emotion]
    for feature in data.columns[:-1]:
        _, p_value = ttest_ind(subset[feature], data[feature])
        if p_value < 0.05:
            num_features[emotion]['significant'] += 1
        else:
            num_features[emotion]['non-significant'] += 1

# Extract emotion labels and corresponding feature counts
emotion_labels = list(num_features.keys())
significant_counts = [num_features[emotion]['significant'] for emotion in emotion_labels]
non_significant_counts = [num_features[emotion]['non-significant'] for emotion in emotion_labels]

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])

In [23]:
X = data.drop('label', axis=1)
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [25]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=70, batch_size=32, verbose=2)

Epoch 1/70
38/38 - 2s - loss: 1.0576 - accuracy: 0.7443 - val_loss: 0.5830 - val_accuracy: 0.8997 - 2s/epoch - 40ms/step
Epoch 2/70
38/38 - 0s - loss: 0.9038 - accuracy: 0.8491 - val_loss: 0.2998 - val_accuracy: 0.9264 - 354ms/epoch - 9ms/step
Epoch 3/70
38/38 - 0s - loss: 0.7421 - accuracy: 0.8600 - val_loss: 0.6670 - val_accuracy: 0.9097 - 351ms/epoch - 9ms/step
Epoch 4/70
38/38 - 0s - loss: 0.5458 - accuracy: 0.8885 - val_loss: 0.3688 - val_accuracy: 0.9030 - 340ms/epoch - 9ms/step
Epoch 5/70
38/38 - 0s - loss: 0.4198 - accuracy: 0.8977 - val_loss: 0.4211 - val_accuracy: 0.9231 - 333ms/epoch - 9ms/step
Epoch 6/70
38/38 - 0s - loss: 0.3982 - accuracy: 0.9028 - val_loss: 0.3264 - val_accuracy: 0.9097 - 364ms/epoch - 10ms/step
Epoch 7/70
38/38 - 0s - loss: 0.2521 - accuracy: 0.9288 - val_loss: 0.3178 - val_accuracy: 0.9264 - 351ms/epoch - 9ms/step
Epoch 8/70
38/38 - 0s - loss: 0.2577 - accuracy: 0.9338 - val_loss: 0.2199 - val_accuracy: 0.9398 - 348ms/epoch - 9ms/step
Epoch 9/70
38/38 

In [27]:
model_acc = model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy: {:.3f}%".format(model_acc * 100))

Test Accuracy: 96.719%


In [28]:
model.save('my_cnn_model.h5')

C:\Users\elyas\anaconda33\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model loaded successfully.
Test Accuracy: 96.719%
1/1 [==============================] - 0s 127ms/step
{'sentiment': 1, 'eeg_data': [0.4338324766675109, 0.33896947569157654, 0.413316920434166, 0.6222413735957005, 0.8584952758218507]}
